# Pockitect Colab Training (Unsloth)

This notebook fine-tunes the model on a GPU-backed Colab runtime using the project training data.

**Before you start:** Runtime → Change runtime type → **GPU**

## 1) Clone the repo

In [ ]:
%cd /content
!rm -rf pockitect-mvp
!git clone https://github.com/BrendenKennedy/pockitect-mvp.git
%cd /content/pockitect-mvp

## 2) Install dependencies

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -r requirements.txt

## 3) (Optional) Validate training data

In [ ]:
!python tools/validate_training_data.py

## 4) (Optional) Regenerate JSONL if you changed training data

In [ ]:
!python tools/convert_to_finetuning_format.py --format unsloth

## 5) Train

In [ ]:
# Memory-optimized for T4 (15GB VRAM)
!python tools/finetune_model.py \
    --model unsloth/Qwen2.5-3B \
    --train data/finetuning/train_unsloth.jsonl \
    --epochs 3 \
    --batch-size 1 \
    --gradient-accumulation 8 \
    --max-seq-len 1024 \
    --early-stopping \
    --early-stopping-patience 3 \
    --eval-steps 25

## 6) Export to GGUF for Ollama

This merges the LoRA adapter with the base model and exports to GGUF format for use with Ollama.

In [ ]:
from unsloth import FastLanguageModel

# Load the fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="data/finetuning/output/adapter",
    max_seq_length=1024,
    dtype=None,
    load_in_4bit=True,
)

# Save as GGUF (Q4_K_M is a good balance of size and quality)
model.save_pretrained_gguf(
    "data/finetuning/output/gguf",
    tokenizer,
    quantization_method="q4_k_m"
)
print("✅ GGUF export complete!")

In [ ]:
## 7) Download the GGUF model

In [ ]:
# Find and download the GGUF file
import glob
from google.colab import files

gguf_files = glob.glob("data/finetuning/output/gguf/*.gguf")
if gguf_files:
    print(f"Downloading: {gguf_files[0]}")
    files.download(gguf_files[0])
else:
    print("No GGUF file found. Check the export step.")

## 8) Create Ollama model locally

After downloading the GGUF file, run on your local machine:

```bash
# Create a Modelfile
echo 'FROM ./pockitect-unsloth.Q4_K_M.gguf' > Modelfile
echo 'PARAMETER temperature 0.2' >> Modelfile

# Create the Ollama model
ollama create pockitect:finetuned -f Modelfile

# Test it
ollama run pockitect:finetuned "I need a simple blog server"
```